In [1]:
import numpy as np

In [2]:
pi = np.array([0.5,0.5])
# a_ij = a[i,j]
a = np.array([[0.7, 0.3],[0.1, 0.9]])
# b_i(o_j) = b[i,j]
b = np.array([[0.05, 0.15, 0.80],[0.75, 0.15, 0.10]])
states = ['hot','cold']
observations = ['1 solo', '2 solos', '3 solos']

input_seq = [2,2,0]

In [21]:
T = len(input_seq)
N = len(pi)

# Initialisation
alpha = [[pi[i]*b[i,input_seq[0]] for i in range(N)]]
psi = [['' for i in range(N)]]
print('\n'.join([f'delta_1({states[i]:s}) = {pi[i]} x {b[i,input_seq[0]]} = {alpha[0][i]}' for i in range(N)]))
print('psi_1(i) = 0 for all i \n')

# Induction
for t in range(T-1):
    temp_alpha = []
    temp_psi = []
    for i in range(N):
        alpha_a = [alpha[t][j]*a[j,i] for j in range(N)]
        s = np.max(alpha_a)
        bi = b[i,input_seq[t+1]]
        temp_psi.append(np.argmax(alpha_a))
        temp_alpha.append(s*bi)
        print(f'delta_{t+2:d}({states[i]:s}) = max[{alpha[t][0]:.7g} x {a[0,i]:.7g}, {alpha[t][1]:.7g} x {a[1,i]:.7g}] x {b[i,input_seq[t+1]]:.7g} = {temp_alpha[i]:.7g}')
        print(f'psi_{t+2:d}({states[i]:s}) = argmax[{alpha[t][0]:.7g} x {a[0,i]:.7g}, {alpha[t][1]:.7g} x {a[1,i]:.7g}] = {states[temp_psi[i]]:s}\n')
    alpha.append(temp_alpha)
    psi.append(temp_psi)

# Termination
q = []
P_best = np.max(alpha[T-1])
q = np.argmax(alpha[T-1])
print(f'P_best('+ ', '.join([observations[i] for i in input_seq]) + f'|mu) = ' + 'max(' + ', '.join([f'{i:.7g}' for i in alpha[T-1]]) + f')' + f' = {P_best}')
print(f'q*_T = ' + 'argmax(' + ', '.join([f'{i:.7g}' for i in alpha[T-1]]) + f')' + f' = {states[q]}\n')

print('Backtrackng:')
qs = [0 for i in range(T)]
qs[-1] = q
for t in range(T-2,-1,-1):
    qs[t] = psi[t+1][qs[t+1]]
    print(f'q*_{t+1} = psi_{t+2}({states[qs[t+1]]:s}) = {states[qs[t]]:s}')

print(f'\nSo the most probable sequence will be ' + ','.join([states[i] for i in qs]))

delta_1(hot) = 0.5 x 0.8 = 0.4
delta_1(cold) = 0.5 x 0.1 = 0.05
psi_1(i) = 0 for all i 

delta_2(hot) = max[0.4 x 0.7, 0.05 x 0.1] x 0.8 = 0.224
psi_2(hot) = argmax[0.4 x 0.7, 0.05 x 0.1] = hot

delta_2(cold) = max[0.4 x 0.3, 0.05 x 0.9] x 0.1 = 0.012
psi_2(cold) = argmax[0.4 x 0.3, 0.05 x 0.9] = hot

delta_3(hot) = max[0.224 x 0.7, 0.012 x 0.1] x 0.05 = 0.00784
psi_3(hot) = argmax[0.224 x 0.7, 0.012 x 0.1] = hot

delta_3(cold) = max[0.224 x 0.3, 0.012 x 0.9] x 0.75 = 0.0504
psi_3(cold) = argmax[0.224 x 0.3, 0.012 x 0.9] = hot

P_best(3 solos, 3 solos, 1 solo|mu) = max(0.00784, 0.0504) = 0.0504
q*_T = argmax(0.00784, 0.0504) = cold

Backtrackng:
q*_2 = psi_3(cold) = hot
q*_1 = psi_2(hot) = hot

So the most probable sequence will be hot,hot,cold
